## Урок 7. Сверточные нейронные сети для анализа текста


### Задание
Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации

2. Рассмотреть 2-а варианта сеточек

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше



In [2]:
!pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=fb954ab3a844fd544862fb959a995a4412d691b92db126f00b7b62fb28e23976
  Stored in directory: /Users/Olga/Library/Caches/pip/wheels/da/d8/66/395317506a23a9d1d7de433ad6a7d9e6e16aab48cf028a0f60
Successfully built stop_words


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.probability import FreqDist

import numpy as np
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import SparseCategoricalCrossentropy
from keras.callbacks import EarlyStopping  

from sklearn.preprocessing import LabelEncoder
import gensim

[nltk_data] Downloading package punkt to /Users/Olga/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [4]:
import keras.backend as K
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [5]:
df = pd.read_excel('отзывы за лето.xls')
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [6]:
df['Rating'].value_counts()

5    14586
1     2276
4     2138
3      911
2      748
Name: Rating, dtype: int64

In [7]:
df_train, df_test = train_test_split(df, test_size=0.33, random_state=42)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((13841, 3), (6818, 3))

In [8]:
df_train

,Rating,Content,Date
0,5,Наконец-то исправили эту чушь с неоргинальной ...,2017-08-09
1,5,Удобно в использовании,2017-07-27
2,5,Отлично,2017-08-08
3,5,Класс,2017-07-25
4,5,Удобно,2017-07-08
...,...,...,...
13836,4,Все нравится,2017-07-29
13837,5,Очень смешно программа пугается рута :),2017-07-28
13838,1,Не могу скачать ошибка номер 24,2017-08-06
13839,5,Сбербанк всегда рядом,2017-08-12


In [9]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [10]:
df_train.head()

,Rating,Content,Date
0,5,наконецтый исправить чушь снеоргинальный проши...,2017-08-09
1,5,удобно использование,2017-07-27
2,5,отлично,2017-08-08
3,5,класс,2017-07-25
4,5,удобно,2017-07-08


In [11]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()
train_corpus

'наконецтый исправить чушь снеоргинальный прошивка приложение удобно пользоваться удобно использование отлично класс удобно замечательный приложение довольный влиять работа устройство приходиться периодически отключать оплатить мобильный приложение коммунальный услуга гораздо простой ноут шаблон проводиться клик удобно сказать пропасть переводить другой клиент сбербанк оплачивать чужой мобильный телефон программа серьёзно поработать отлично смело качать устраивать принцип хороший программа удобно быстро обновление сталон удобно делать выписка счёт тк увидеть поступление сегодняшний число происходить день просить разработчик обратить особый внимание удобно норма отлично антивирус клавиатура пятизначный пин отлично пойти класс asus ze551ml клёво работать антивирус справляться 10 сечь довольный пятёрочка  быстро удобно сбербанк онлайн удобство комфорт хороший приложение правда многий функциямин пользоваться нравиться разработчик удобно нравиться норма нравиться заходить какуюнеделюисправи

In [12]:
tokens = word_tokenize(train_corpus)
tokens

['наконецтый',
 'исправить',
 'чушь',
 'снеоргинальный',
 'прошивка',
 'приложение',
 'удобно',
 'пользоваться',
 'удобно',
 'использование',
 'отлично',
 'класс',
 'удобно',
 'замечательный',
 'приложение',
 'довольный',
 'влиять',
 'работа',
 'устройство',
 'приходиться',
 'периодически',
 'отключать',
 'оплатить',
 'мобильный',
 'приложение',
 'коммунальный',
 'услуга',
 'гораздо',
 'простой',
 'ноут',
 'шаблон',
 'проводиться',
 'клик',
 'удобно',
 'сказать',
 'пропасть',
 'переводить',
 'другой',
 'клиент',
 'сбербанк',
 'оплачивать',
 'чужой',
 'мобильный',
 'телефон',
 'программа',
 'серьёзно',
 'поработать',
 'отлично',
 'смело',
 'качать',
 'устраивать',
 'принцип',
 'хороший',
 'программа',
 'удобно',
 'быстро',
 'обновление',
 'сталон',
 'удобно',
 'делать',
 'выписка',
 'счёт',
 'тк',
 'увидеть',
 'поступление',
 'сегодняшний',
 'число',
 'происходить',
 'день',
 'просить',
 'разработчик',
 'обратить',
 'особый',
 'внимание',
 'удобно',
 'норма',
 'отлично',
 'антивирус',
 

In [13]:
tokens_filtered = [word for word in tokens if word.isalnum()]

dist = FreqDist(tokens_filtered)
dist

FreqDist({'приложение': 4123, 'удобно': 2201, 'работать': 1288, 'удобный': 1182, 'отлично': 860, 'нравиться': 763, 'хороший': 681, 'отличный': 677, 'телефон': 627, 'супер': 540, ...})

In [14]:
max_words = 200

tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [15]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}
vocabulary

{'приложение': 1,
 'удобно': 2,
 'работать': 3,
 'удобный': 4,
 'отлично': 5,
 'нравиться': 6,
 'хороший': 7,
 'отличный': 8,
 'телефон': 9,
 'супер': 10,
 'быстро': 11,
 'обновление': 12,
 'пароль': 13,
 'мочь': 14,
 'пользоваться': 15,
 'антивирус': 16,
 'банк': 17,
 'вход': 18,
 'устраивать': 19,
 'сбербанк': 20,
 'раз': 21,
 'прошивка': 22,
 'карта': 23,
 'проблема': 24,
 'рута': 25,
 'программа': 26,
 'ошибка': 27,
 'разработчик': 28,
 'сделать': 29,
 'приходиться': 30,
 'вводить': 31,
 'перевод': 32,
 'счёт': 33,
 'писать': 34,
 'норма': 35,
 'деньга': 36,
 'довольный': 37,
 'около': 38,
 'постоянно': 39,
 'нормально': 40,
 'код': 41,
 'исправить': 42,
 'смс': 43,
 'платёж': 44,
 'понятно': 45,
 'последний': 46,
 'функция': 47,
 'зайти': 48,
 'свой': 49,
 'вылетать': 50,
 'мобильный': 51,
 'стать': 52,
 'шаблон': 53,
 'приходить': 54,
 'возможность': 55,
 'право': 56,
 'делать': 57,
 'иня': 58,
 'проверка': 59,
 'класс': 60,
 'установить': 61,
 'root': 62,
 'заходить': 63,
 '5': 

In [16]:
max_len = 40

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [17]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)

In [18]:
x_train

array([[  0,   0,   0, ...,   1,   2,  15],
       [  0,   0,   0, ...,   0,   2, 181],
       [  0,   0,   0, ...,   0,   0,   5],
       ...,
       [  0,   0,   0, ..., 164,  27,  84],
       [  0,   0,   0, ...,   0,   0,  20],
       [  0,   0,   0, ...,   0, 113,   5]], dtype=int32)

In [19]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(df_train['Rating']) 
test_enc_labels = le.transform(df_test['Rating'])
le.classes_

array([1, 2, 3, 4, 5])

In [20]:
train_enc_labels

array([4, 4, 4, ..., 0, 4, 4])

In [21]:
num_classes = 5
y_train = tf.keras.utils.to_categorical(train_enc_labels, num_classes=num_classes)
y_test = tf.keras.utils.to_categorical(test_enc_labels, num_classes=num_classes)
y_train

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.]], dtype=float32)

In [22]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len)) 

model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))


In [23]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[get_f1])

In [24]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

2022-07-23 16:42:36.431801: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-07-23 16:42:36.431847: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-07-23 16:42:36.434095: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-07-23 16:42:37.255254: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20
 1/25 [>.............................] - ETA: 41s - loss: 1.6127 - get_f1: 0.0000e+00

2022-07-23 16:42:39.139808: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-07-23 16:42:39.139836: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.


 2/25 [=>............................] - ETA: 5s - loss: 1.5980 - get_f1: 0.0000e+00 

2022-07-23 16:42:39.355962: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-07-23 16:42:39.379810: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-07-23 16:42:39.401782: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/train/plugins/profile/2022_07_23_16_42_39

2022-07-23 16:42:39.405701: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ./logs/train/plugins/profile/2022_07_23_16_42_39/MacBook-Air-Olga.local.trace.json.gz
2022-07-23 16:42:39.426208: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/train/plugins/profile/2022_07_23_16_42_39

2022-07-23 16:42:39.426777: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to ./logs/train/plugins/profile/2022_07_23_16_42_39/MacBook-Air-Olga.local.memory_profile.json.gz
2

25/25 [==============================] - 8s 244ms/step - loss: 1.3673 - get_f1: 0.1356 - val_loss: 1.1894 - val_get_f1: 0.7005


In [25]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 1s 75ms/step - loss: 1.1623 - get_f1: 0.7151


Test loss: 1.1623296737670898
Test f1_score: 0.7151089310646057


In [36]:
#http://vectors.nlpl.eu/repository/
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('./180/model.bin', binary=True)  
len(word_vectors), len(word_vectors[1]), word_vectors[1]

(189193,
 300,
 array([ 0.56827927, -0.3209115 , -0.01912439, -1.9356475 , -0.8124491 ,
         1.657973  , -1.3159128 ,  1.1186373 , -2.0223322 , -3.2186239 ,
        -0.8087832 , -1.6359693 ,  0.30781016, -1.9637547 ,  1.1508301 ,
         0.09154594,  1.499977  , -1.9233339 , -1.395698  ,  0.4522272 ,
         1.3875449 , -0.6991994 ,  1.1241058 , -0.8872212 ,  1.5669478 ,
         0.10525192, -0.72891355, -1.7446259 , -0.39187282, -0.21783888,
         0.99993414, -2.8352168 ,  0.28585172, -1.2888701 , -0.7390094 ,
         0.7050435 ,  2.6338942 ,  0.37333074,  2.078776  , -0.19290456,
        -0.73848265, -1.7742666 ,  1.6497376 ,  3.7614036 , -1.2978705 ,
        -0.00728586,  1.3725481 , -1.5491337 , -0.81334364, -0.7748071 ,
        -0.61844414,  0.7076229 ,  0.53189814, -0.519196  ,  1.2863954 ,
         0.99337226,  1.1154282 , -0.930117  ,  1.572236  , -1.2262355 ,
        -0.86334157,  0.61980397, -0.09515494,  0.5440264 , -0.50054914,
         0.32205996, -1.49439   ,  0

In [37]:
word_vectors_matrix = [word_vectors[i][:128] for i in range(200)]
word_vectors_matrix

[array([ 1.0967804e+00, -2.2944486e+00,  1.9791678e+00,  3.4804371e-01,
         4.0753922e-01,  1.3286122e+00, -9.3368673e-01,  5.4947221e-01,
        -6.8077618e-01, -7.4963701e-01, -8.0936104e-02,  6.5788299e-02,
        -8.8564938e-01,  5.0793958e-01, -1.0864110e+00, -4.3317631e-01,
         2.0482888e-02, -5.7119979e-03, -1.0036458e+00,  3.1729680e-01,
         1.1956499e+00,  1.0685917e+00, -8.9309484e-01,  7.4419886e-01,
         4.4171312e-01, -2.0080043e-01, -2.6623638e+00,  1.7598321e-01,
        -2.0019765e+00, -5.6796205e-01, -2.0350738e-01, -7.3997623e-01,
         8.0226004e-01,  1.4174094e+00,  1.0990121e-01,  1.2313192e+00,
         1.6655500e+00, -2.6187131e-01,  1.7604357e+00, -3.1903556e-01,
         2.4266930e+00, -1.2458172e+00,  1.0440445e+00,  2.7978971e+00,
        -8.2489556e-01,  4.3603179e-01,  9.2311478e-01, -1.8360819e-01,
        -6.3582733e-02,  2.9669294e-01,  5.6388801e-01,  2.5535300e+00,
         2.3046949e+00, -1.8354168e+00, -6.8992484e-01, -1.38589

In [38]:
initializer = tf.keras.initializers.Constant(word_vectors_matrix)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, embeddings_initializer =initializer, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [39]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[get_f1])

In [40]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

epochs = 20
batch_size = 512

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

2022-07-23 17:43:58.427545: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-07-23 17:43:58.428237: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-07-23 17:43:58.435250: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.


Epoch 1/20
 2/25 [=>............................] - ETA: 5s - loss: 1.5561 - get_f1: 0.5274 

2022-07-23 17:44:00.721624: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-07-23 17:44:00.721828: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-07-23 17:44:00.996098: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-07-23 17:44:01.009287: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-07-23 17:44:01.032499: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/train/plugins/profile/2022_07_23_17_44_01

2022-07-23 17:44:01.036190: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to ./logs/train/plugins/profile/2022_07_23_17_44_01/MacBook-Air-Olga.local.trace.json.gz
2022-07-23 17:44:01.069938: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: ./logs/train/plugins/profile/2022_07_23_17_44_01

2022-07-23 17

25/25 [==============================] - 7s 237ms/step - loss: 0.9939 - get_f1: 0.6956 - val_loss: 0.8325 - val_get_f1: 0.7205


In [41]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test f1_score:', score[1])

14/14 [==============================] - 1s 64ms/step - loss: 0.8248 - get_f1: 0.7317


Test loss: 0.8247653841972351
Test f1_score: 0.7317059636116028
